In [ ]:
!pip install -qU langchain_community langchain_nvidia_ai_endpoints python-dotenv PyPDF faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.7 MB/s eta 0:00:00


In [ ]:
import os
import os

# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv()) # read .env file
# apikey = os.getenv('NVIDIA_API_KEY')

# If you run in Google Colab
from google.colab import userdata
apikey = userdata.get('NVIDIA_API_KEY')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [ ]:
# download BOT 2566 annual report from https://www.bot.or.th/content/dam/bot/documents/th/thai-economy/state-of-thai-economy/annual-report/annual-econ-report-th-2566.pdf
!wget https://www.bot.or.th/content/dam/bot/documents/th/thai-economy/state-of-thai-economy/annual-report/annual-econ-report-th-2566.pdf -O annual-econ-report-th-2566.pdf

PDF_FILE = "annual-econ-report-th-2566.pdf"

--2024-08-18 05:03:07--  https://www.bot.or.th/content/dam/bot/documents/th/thai-economy/state-of-thai-economy/annual-report/annual-econ-report-th-2566.pdf
Resolving www.bot.or.th (www.bot.or.th)... 23.209.37.11, 23.209.36.235, 2600:1408:5400:d::170c:93a5, ...
Connecting to www.bot.or.th (www.bot.or.th)|23.209.37.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 342845 (335K) [application/pdf]
Saving to: ‘annual-econ-report-th-2566.pdf’

annual-econ-report- 100%[===================>] 334.81K   285KB/s    in 1.2s    

2024-08-18 05:03:11 (285 KB/s) - ‘annual-econ-report-th-2566.pdf’ saved [342845/342845]



In [ ]:
!ls -l /content/drive/MyDrive/LLM/G-ABLE-Update/Presales_Update-2024-08/images/documents.png

-rw------- 1 root root 157038 Aug 17 15:15 /content/drive/MyDrive/LLM/G-ABLE-Update/Presales_Update-2024-08/images/documents.png


## Loading the PDF document

Let's start by loading the PDF document and breaking it down into separate pages.

<img src='images/documents.png' width="1000">

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 7
Length of a page: 2988
Content of a page:  
 
1 
 สรุปภาวะเศรษฐกิจการเงินต่างประเทศและไทย ปี 2566 และแนวโน้ม  
เศรษฐกิจต่างประเทศปี 2566 และแนวโน้ม  
เศรษฐกิจโลกปี 256 6 ขยายตัวในอัตราที่ชะลอลงจากปีก่อน โดยการบริโภคภาคเอกชน  
โดยเฉพาะภาคบริการ ยังเป็นแรงขับเคลื่อนเศรษฐกิจที่สำคัญในหลายประเทศ จากการผ่อนคลายมาตรการ
ควบคุมการเดินทางระหว่างประเทศ และนโยบายการเปิดประเทศของจีน ในช่วงต้นปี  ขณะที่ปริมาณการค้า
โลกที่ชะลอตัว ส่งผลให้การผลิตภาคอุตสาหกรรมและการส่งออกสินค้าใน หลายประเทศ โดยเฉพาะกลุ่ม
ประเทศเอเชีย ชะลอลง ซึ่งเป็นผลจาก  (1) การดำเนินนโยบายการเงินที่เข้มงวดของธนาคารกลาง หลาย
ประเทศเพื่อดูแลเงินเฟ้อที่มีแนวโน้มทรงตัวในระดับสูงนานกว่าคาด ซึ่งส่งผลให้สถาบันการเงินบางแห่งใน
สหรัฐฯ และยุโรปประสบปัญหาด้านสภาพคล่อง ในไตรมาส 1 อย่างไรก็ดี ผลต่อเศรษฐกิจและการเงินโลก
ค่อนข้างจำกัด เนื่องจากทางการได้ดำเนินมาตรการสกัดผลกระทบที่จะลุกลามเป็นวงกว้างได้อย่างรวดเร็ว 
อาทิ การค้ำประกันเงินฝาก และการตั้งกองทุนเพื่อเสริมสภาพคล่องแก่สถาบันการเงิน เป็นต้น  (2) การฟื้นตัว
ของเศรษฐกิจจีนท

## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 24
Length of a chunk: 919
Content of a chunk: 1 
 สรุปภาวะเศรษฐกิจการเงินต่างประเทศและไทย ปี 2566 และแนวโน้ม  
เศรษฐกิจต่างประเทศปี 2566 และแนวโน้ม  
เศรษฐกิจโลกปี 256 6 ขยายตัวในอัตราที่ชะลอลงจากปีก่อน โดยการบริโภคภาคเอกชน  
โดยเฉพาะภาคบริการ ยังเป็นแรงขับเคลื่อนเศรษฐกิจที่สำคัญในหลายประเทศ จากการผ่อนคลายมาตรการ
ควบคุมการเดินทางระหว่างประเทศ และนโยบายการเปิดประเทศของจีน ในช่วงต้นปี  ขณะที่ปริมาณการค้า
โลกที่ชะลอตัว ส่งผลให้การผลิตภาคอุตสาหกรรมและการส่งออกสินค้าใน หลายประเทศ โดยเฉพาะกลุ่ม
ประเทศเอเชีย ชะลอลง ซึ่งเป็นผลจาก  (1) การดำเนินนโยบายการเงินที่เข้มงวดของธนาคารกลาง หลาย
ประเทศเพื่อดูแลเงินเฟ้อที่มีแนวโน้มทรงตัวในระดับสูงนานกว่าคาด ซึ่งส่งผลให้สถาบันการเงินบางแห่งใน
สหรัฐฯ และยุโรปประสบปัญหาด้านสภาพคล่อง ในไตรมาส 1 อย่างไรก็ดี ผลต่อเศรษฐกิจและการเงินโลก
ค่อนข้างจำกัด เนื่องจากทางการได้ดำเนินมาตรการสกัดผลกระทบที่จะลุกลามเป็นวงกว้างได้อย่างรวดเร็ว 
อาทิ การค้ำประกันเงินฝาก และการตั้งกองทุนเพื่อเสริมสภาพคล่องแก่สถาบันการเงิน เป็นต้น  (2) การฟื้นตัว


## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embeddings = NVIDIAEmbeddings(
    model="nvidia/nv-embedqa-e5-v5",
    api_key=apikey,
    truncate="NONE"
)

vectorstore = FAISS.from_documents(chunks, embeddings)

## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("แนวโน้มการขยายตัวทางเศรษฐกิจของปี 2566 เป็นอย่างไรเมื่อเทียบกับปีก่อน")

[Document(metadata={'source': 'annual-econ-report-th-2566.pdf', 'page': 5}, page_content='5 \n  \nรวมเหตุการณ์สำคัญที่กระทบเศรษฐกิจการเงินโลกและไทย  \n \n \n \n \n \n \n \n \n \nปี 2566 เป็นอีกปีที่มีเหตุการณ์สำคัญ ที่ส่งผลกระทบต่อระบบเศรษฐกิจการเงินของโลกและไทย ดังนี้  \n1. จีนได้ประกาศเปิดประเทศในวันที่ 8 ม.ค. 2566 โดยอนุญาตให้ชาวจีนเดินทางออกนอกประเทศได้\nโดยไม่ต้องตรวจเชื้อโควิดและกักตัว หลังจากนั้นในวันที่ 15 มี.ค. 2566 จีนประกาศอนุญาตให้\nนักท่องเที่ยวต่างชาติเดินทางกลับไปเที่ยวจีนได้  \n2. วันที่ 12 มี.ค. 2566 ทางการสหรัฐประกาศปิดกิจการ  Signature Bank หลังจากนั้นในวันที่ 17 มี .ค. \n2566 ธนาคาร Silicon Valley (SVB) ประกาศล้มละลาย  นอกจากนี้  ในวันที่ 19 มี.ค. 2566 Union \nBank of Switzerland  (UBS) ซื้อกิจการของ Credit Suis se ในราคากว่า 3 พันล้านฟรังก์ โดยได้รับ\nความสนับสนุนจากธนาคารกลาง และรัฐบาลสวิตเซอร์แลนด์  \n3. การเลือกตั้งสมาชิกสภาผู้แทนราษฎรไทยเป็นการทั่วไปครั้งที่ 27 จัดขึ้นเมื่อวันที่ 14 พฤษภาคม 2566  \nและมีการแถลงข่าวการจัดตั้งรัฐบาล ในวันที่ 21 ส.ค. 2566 โดยพรรคเ

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# llm model
model = ChatNVIDIA(model="meta/llama-3.1-8b-instruct",
                   api_key=apikey,
                   top_k=3,
                   temperature=0.0, max_tokens=2000)  # Increase number of max_tokens for Thai

model.invoke("อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566")

AIMessage(content='อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566 ได้แก่\n\n1. อุตสาหกรรมเทคโนโลยี: อุตสาหกรรมเทคโนโลยี เช่น เทคโนโลยีสารสนเทศ, เทคโนโลยีเครื่องจักร, เทคโนโลยีชีวภาพ, เทคโนโลยีพลังงาน, เทคโนโลยีการสื่อสารมวลชน เป็นต้น คาดว่าจะมีการขยายตัวอย่างมากในปี 2566 เนื่องจากความต้องการเทคโนโลยีที่ทันสมัยและความสามารถในการปรับตัวเข้ากับเทคโนโลยีใหม่ๆ\n\n2. อุตสาหกรรมการแพทย์: อุตสาหกรรมการแพทย์ เช่น อุตสาหกรรมผลิตยา, อุตสาหกรรมผลิตอุปกรณ์ทางการแพทย์, อุตสาหกรรมผลิตเครื่องมือทางการแพทย์ เป็นต้น คาดว่าจะมีการขยายตัวในปี 2566 เนื่องจากความต้องการด้านสุขภาพที่เพิ่มขึ้นและความสามารถในการพัฒนาเทคโนโลยีการแพทย์ที่ทันสมัย\n\n3. อุตสาหกรรมการเกษตร: อุตสาหกรรมการเกษตร เช่น อุตสาหกรรมผลิตอาหาร, อุตสาหกรรมผลิตผลิตภัณฑ์จากพืช, อุตสาหกรรมผลิตผลิตภัณฑ์จากสัตว์ เป็นต้น คาดว่าจะมีการขยายตัวในปี 2566 เนื่องจากความต้องการอาหารที่เพิ่มขึ้นและความสามารถในการพัฒนาเทคโนโลยีการเกษตรที่ทันสมัย\n\n4. อุตสาหกรรมการขนส่ง: อุตสาหกรรมการขนส่ง เช่น อุตสาหกรรมรถยนต์, อุตสาหกรรมเครื่องบิน, อุตสาหกรรมเรือ เป็นต้น คาดว

In [ ]:
model

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='meta/llama-3.1-8b-instruct', temperature=0.0, max_tokens=2000)

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
print(chain.invoke("อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566"))

อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566 ได้แก่

1. อุตสาหกรรมเทคโนโลยี: อุตสาหกรรมเทคโนโลยี เช่น เทคโนโลยีสารสนเทศ, เทคโนโลยีเครื่องจักร, เทคโนโลยีชีวภาพ, เทคโนโลยีพลังงาน, เทคโนโลยีการสื่อสารมวลชน เป็นต้น คาดว่าจะมีการขยายตัวอย่างมากในปี 2566 เนื่องจากความต้องการเทคโนโลยีที่ทันสมัยและความสามารถในการปรับตัวเข้ากับเทคโนโลยีใหม่ๆ

2. อุตสาหกรรมการแพทย์: อุตสาหกรรมการแพทย์ เช่น อุตสาหกรรมผลิตยา, อุตสาหกรรมผลิตอุปกรณ์ทางการแพทย์, อุตสาหกรรมผลิตเครื่องมือทางการแพทย์ เป็นต้น คาดว่าจะมีการขยายตัวในปี 2566 เนื่องจากความต้องการด้านสุขภาพที่เพิ่มขึ้นและความสามารถในการพัฒนาเทคโนโลยีการแพทย์ที่ทันสมัย

3. อุตสาหกรรมการเกษตร: อุตสาหกรรมการเกษตร เช่น อุตสาหกรรมผลิตอาหาร, อุตสาหกรรมผลิตผลิตภัณฑ์จากพืช, อุตสาหกรรมผลิตผลิตภัณฑ์จากสัตว์ เป็นต้น คาดว่าจะมีการขยายตัวในปี 2566 เนื่องจากความต้องการอาหารที่เพิ่มขึ้นและความสามารถในการพัฒนาเทคโนโลยีการเกษตรที่ทันสมัย

4. อุตสาหกรรมการขนส่ง: อุตสาหกรรมการขนส่ง เช่น อุตสาหกรรมรถยนต์, อุตสาหกรรมเครื่องบิน, อุตสาหกรรมเรือ เป็นต้น คาดว่าจะมีการขยายตัวในปี 2566 เ

## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.

<img src='images/prompt.png' width="1000">

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context.

Alwasy answer the question based on the context in Thai. If you can't answer the
question, reply "ฉันมีข้อมูลไม่เพียงพอ".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="นี่คือ context", question="นีคือคำถาม"))


You are an assistant that provides answers to questions based on
a given context.

Alwasy answer the question based on the context in Thai. If you can't answer the
question, reply "ฉันมีข้อมูลไม่เพียงพอ".

Be as concise as possible and go straight to the point.

Context: นี่คือ context

Question: นีคือคำถาม



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [ ]:
chain = prompt | model | parser

chain.invoke({
    "context": "Anna's sister is Susan",
    "question": "Who is Susan's sister?"
})


'แอนนา'

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [ ]:
questions = [
    "อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566",
    "จงสรุปภาพรวมด้านอัตราเงินเฟ้อและเศรษฐกิจภาคครัวเรือน",
    "แนวโน้มเศรษฐกิจไทยในอนาคตจะเป็นอย่างไร",
    "ความเสี่ยงในอนาคตที่ไทยควรเตรียมรับมือ",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566
Answer: ภาคบริการ
*************************

Question: จงสรุปภาพรวมด้านอัตราเงินเฟ้อและเศรษฐกิจภาคครัวเรือน
Answer: ภาพรวมด้านอัตราเงินเฟ้อและเศรษฐกิจภาคครัวเรือนในปี 2566 มีดังนี้

* ดัชนีราคาผู้บริโภคพื้นฐาน (ไม่รวมราคาอาหารสดและพลังงาน) เพิ่มขึ้น 2.51%
* อัตราการว่างงานอยู่ที่ 1.3%
* หนี้สาธารณะ ณ สิ้นงวดอยู่ที่ 10,587 พันล้านบาท
* สัดส่วนหนี้สาธารณะต่อ GDP อยู่ที่ 60.9%
*************************

Question: แนวโน้มเศรษฐกิจไทยในอนาคตจะเป็นอย่างไร
Answer: แนวโน้มเศรษฐกิจไทยในอนาคตมีความไม่แน่นอนและผันผวน เนื่องจากมีปัจจัยหลายอย่างที่มีอิทธิพล เช่น ความขัดแย้งด้านภูมิรัฐศาสตร์ เศรษฐกิจและตลาดการเงินโลกที่มีความไม่แน่นอน และความสามารถในการเร่งเบิกจ่ายของภาครัฐหลังงบประมาณปี 2567
*************************

Question: ความเสี่ยงในอนาคตที่ไทยควรเตรียมรับมือ
Answer: ความเสี่ยงในอนาคตที่ไทยควรเตรียมรับมือ ได้แก่ ความไม่แน่นอนและผันผวนของเศรษฐกิจและตลาดการเงินโลก, ความขัดแย้งด้านภูมิรัฐศาสตร์, และความสามารถในการเร่งเบิกจ่ายของภาครัฐหลังงบประ